### 손글씨 이미지 데이터 MNIST

- 인코딩 된 바이너리 데이터를 디코딩하여 처리하는 방식 확인
- 지도 학습
- 학습용 데이터는 60000개, 테스트 데이터는 10000개
- 결론
    - 학습 후 새로운 데이터를 입력시 판별
    - 0~9까지의 손글씨 이미지를 판별
    - 데이터는 url을 직접 획득해서, 원하는 곳에 다운로드 시키겠다.

### 절차

|No|단계|내용|
|:--:|:--|:--|
|1|연구목표|- 손글씨 이미지 (0-9)를 학습시켜서 새로운 손글씨 이미지를 판별해 내는 머신러닝 모델을 구축<br>- 압축된 이미지를 압축해제<br>- 인코딩 된 데이터를 디코딩 처리<br>- 28x28로 구성된 픽셀 이미지 데이터를 벡터화 처리<br>- 시스템 통합의 결과를 보고 연구 목표를 설정해야 하는데 시스템 통합은 생략이므로 이 부분 생략|
|2|데이터획득/수집| - http://yann.lecun.com/exdb/mnist/ 접속<br>- Web Scraping을 통해서 데이터의 URL 획득<br>- 지정된 위치에 다운로드 -> 압축해제|
|3|데이터준비/전처리|- 디코딩(내부구조를 알 수 있는 인코딩 문서(MNIST Database) 필요)<br>- 에디언(Endian) 처리<br>- 벡터화 처리|
|4|데이터탐색/통찰/시각화분석|- skip|
|5|데이터모델링(머신러닝모델링)|- 분류 알고리즘 사용<br>- 알고리즘 선택 -> 훈련용 데이터와 테스트용 데이터를 나눔 -> 학습 -> 예측 -> 평가|
|6|시스템통합|- skip|

#### 2. 데이터획득/수집

- 모듈 준비

In [1]:
import urllib.request as req
from bs4 import BeautifulSoup

 - web scraping

In [2]:
rootUrl = 'http://yann.lecun.com/exdb/mnist/'
soup = BeautifulSoup(req.urlopen(rootUrl), 'html5lib')

- train-images-idx3-ubyte.gz, ... 등 총 4개 url 획득 

In [3]:
# http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz

# 모든 요소 tt 중에 상위 4개가 링크
for tt in soup.find_all('tt')[:4]:
    
    # 링크 값이나 링크 문자열이나 현재 동일하다. 문자열 획득으로 처리
    # 링크 최종 주소는 rootUrl과 링크 문자열을 더한 것이다.
    print(tt.a.string)

train-images-idx3-ubyte.gz
train-labels-idx1-ubyte.gz
t10k-images-idx3-ubyte.gz
t10k-labels-idx1-ubyte.gz


In [4]:
# 링크 문자열을 굳이 리스트에 담은 이유는 반복 작업이 예상되었기 때문
files = [tt.a.string for tt in soup.find_all('tt')[:4]]
files

['train-images-idx3-ubyte.gz',
 'train-labels-idx1-ubyte.gz',
 't10k-images-idx3-ubyte.gz',
 't10k-labels-idx1-ubyte.gz']

- 다운로드 -> 압축 해제(반복 작업)

In [5]:
# 필요 모듈
import os, os.path, gzip

In [6]:
# 위치 선정(압축된 파일을 다운로드 할 위치)
savePath = './data/mnist'

# 해당 디렉터리가 없으면 만들어라.
if not os.path.exists(savePath):    # 물리적으로 해당 경로가 없다.
    os.makedirs(savePath)

In [7]:
# 저장
# tqdm : 진행율을 보여주는 모듈
from tqdm import tqdm_notebook

for file in tqdm_notebook(files):
    print('소스', rootUrl + file)
    
    # 저장위치 및 파일명
    local_path = '%s/%s' % (savePath, file)
    print('대상', local_path)
    
    # 웹 상에 존재하는 리소스를 로컬 디스크 상에 직접 저장
    req.urlretrieve(rootUrl + file, local_path)

소스 http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
대상 ./data/mnist/train-images-idx3-ubyte.gz
소스 http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
대상 ./data/mnist/train-labels-idx1-ubyte.gz
소스 http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
대상 ./data/mnist/t10k-images-idx3-ubyte.gz
소스 http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz
대상 ./data/mnist/t10k-labels-idx1-ubyte.gz



In [8]:
# 압축 해제
# 원본 : train-images-idx3-ubyte.gz
# 해제 : train-images-idx3-ubyte <- 원본 파일의 이름을 사용하겠다.
for file in tqdm_notebook(files):
    
    # 원본 파일의 경로
    ori_gzip_file = '%s/%s' % (savePath, file)
    print('%s/%s' % (savePath, file))
    
    # 압축 해제 파일의 경로, 파일명은 다양한 방법으로 획득 가능
    # 파일명에 반드시 확장자가 있을 필요는 없다.
    target_file = '%s/%s' % (savePath, file[:-3])
    print('%s/%s' % (savePath, file[:-3]))
    
    # 압축 해제
    # gzip의 파일 오픈 -> 읽기 -> 쓰기
    with gzip.open(ori_gzip_file, 'rb') as fg:
        
        # 읽기(압축 해제를 수행했다.)
        tmp = fg.read()
        
        # 쓰기(일반 파일로 기록)
        with open(target_file, 'wb') as f:
            f.write(tmp)

./data/mnist/train-images-idx3-ubyte.gz
./data/mnist/train-images-idx3-ubyte
./data/mnist/train-labels-idx1-ubyte.gz
./data/mnist/train-labels-idx1-ubyte
./data/mnist/t10k-images-idx3-ubyte.gz
./data/mnist/t10k-images-idx3-ubyte
./data/mnist/t10k-labels-idx1-ubyte.gz
./data/mnist/t10k-labels-idx1-ubyte



#### 데이터준비/전처리

- 디코딩(내부구조를 알 수 있는 인코딩 문서(MNIST Database) 필요)
- 에디언(Endian) 처리(TCP/IP 상에서 통신 수행 시 중요)
    - 컴퓨터 메모리와 같은 1차원 공간에 여러 개의 연속된 데이터를 배열하는 방법
    - 종류 : 바이트를 배치하는 오더(순서)를 앞에서부터 혹은 뒤에서부터 채우는가?
        - 0x12345678
        - 빅 에디언 : 값을 앞에서부터 채운다.<br>
            0x12, 0x34, 0x56, 0x78
        - 리틀 에디언 :값을 뒤에서부터 채운다.<br>
            0x78, 0x56, 0x34, 0x12
        - 위의 예는 정수값(4byte)을 예로 든 것이고, 단지 값이 어떻게 기록됬는지만 이해하고 그대로 값을 복원할 수 있으면 끝 
- 벡터화 처리

- LABEL FILE
    - magic number : 4byte -> 에디안 체크
    - LABEL 수 : 4byte     -> 에디안 체크
    - LABEL 데이터 : 1byte, ... -> 0 ~ 9 값
    - 크기 = 4 + 4 + LABEL 수 * 1byte = 8 + 60000 = 60004byte
    
- IMAGE FILE
    - magic number : 4byte      -> 에디안 체크
    - 손글씨 이미지 개수 : 4byte  -> 에디안 체크
    - 가로 크기(픽셀 수) : 4byte  -> 에디안 체크
    - 세로 크기(픽셀 수) : 4byte  -> 에디안 체크
    - 각각의 픽셀 값 : unsigned 1 byte(= 8bit)(0 ~ 2^8 - 1 : 0 ~ 255(0xFF))

In [11]:
# 원 데이터의 구조를 이해했으니 구조에 맞춰서 데이터를 디코딩(decoding)한다.
# struct : 바이너리 데이터를 빅/리틀 에디안 방식으로 특정 바이트만큼 읽는 기능 제공
import struct

In [10]:
# 테스트용 레이블 파일 처리
# 바이너리 읽기 모드
label_f = open('./data/mnist/t10k-labels-idx1-ubyte', 'rb')
image_f = open('./data/mnist/t10k-images-idx3-ubyte', 'rb')

# 바이너리 데이터는 헤더부터 읽어서 데이터의 유효성이나 종류를 인지
# MNIST 파일은 규격서에 high-endian(빅 에디안)으로 수치값을 기술했다고 확인되었다.

# 헤더 정보 추출
# label 파일은 헤더가 4 + 4 = 8byte 이다.(규격서 기준)
# high endian -> '>'
# 4 -> I(i의 대문자)
magic_number, label_count = struct.unpack('>II', label_f.read(8))

# magic_number : 2049 -> 레이블 파일이다.
# label_count : 10000 -> 데이터의 개수(레이블의 개수, 답의 개수)
print(magic_number, label_count)

magic_number2, image_count, row, col = struct.unpack('>IIII', image_f.read(16))
print(magic_number2, image_count, row, col)

# 헤더 크기 = 16 + 이미지 1개 데이터 크기(28 * 28) * 총 이미지 개수(10000)
print('이미지 파일의 크기', 4 + 4 + 4 + 4 + 10000 * 28 * 28)

# 헤더 정보를 기초로 반복 작업 수행 : 정답 추출, 이미지 추출
for idx in tqdm_notebook(range(image_count)):
    
    # 정답 추출 : label_f를 통해서 1 byte 읽는다. 단, unsigned(부호 없는, 양수) byte -> 'B'
    # 파일을 읽으면 읽은 만큼 누적으로 커서(파일 포인터) 위치 이동
    label_tmp = struct.unpack('B', label_f.read(1))
    
    # (7,) 이렇게 리턴되서 인덱싱을 통해서 값 획득
    label = label_tmp[0]
    print(label)
    
    # ---------- 여기부터는 아래 셀에서 구현 -----------
    
    # 이미지 추출  
    
    # 이미지 데이터의 벡터화
    
    break
    pass
    
# 닫기
label_f.close()
image_f.close()

2049 10000
2051 10000 28 28
이미지 파일의 크기 7840016


7


In [ ]:
label_f = open('./data/mnist/t10k-labels-idx1-ubyte', 'rb')
image_f = open('./data/mnist/t10k-images-idx3-ubyte', 'rb')

magic_number, label_count = struct.unpack('>II', label_f.read(8))
magic_number2, image_count, row, col = struct.unpack('>IIII', image_f.read(16))

# 이미지 한 개당 크기
pixels = row * col    # 28 x 28

for idx in tqdm_notebook(range(image_count)):
    label_tmp = struct.unpack('B', label_f.read(1))
    label = label_tmp[0]
    
    # 이미지 추출 -> 바이너리 데이터를 읽는다. -> 에디안은 관계 없음
    binaryData = image_f.read(pixels)
#     print(type(binaryData), len(binaryData), binaryData)
    
    # 픽셀값 하나 하나를 문자열로 만들어서 리스트에 담는다.
    strData = list(map(lambda x: str(x), binaryData))
    print(strData)
    
    
    # ---------- 여기부터는 아래 셀에서 구현 -----------
    
    # csv에 한 줄의 데이터로 기록 -> 1 + 784개의 컬럼으로 기록 -> 1개의 데이터 표현 
    # 구분자 ,
    
    # pgm 파일로 dump 처리해서 확인(데이터의 유효성 확인)
    
    # 이미지 데이터의 벡터화
    
    break
    pass

label_f.close()
image_f.close()

In [ ]:
label_f = open('./data/mnist/t10k-labels-idx1-ubyte', 'rb')
image_f = open('./data/mnist/t10k-images-idx3-ubyte', 'rb')

# csv 파일 오픈
csv_f = open('./data/mnist/t10k.csv', 'w', encoding='utf-8', newline='')

magic_number, label_count = struct.unpack('>II', label_f.read(8))
magic_number2, image_count, row, col = struct.unpack('>IIII', image_f.read(16))

pixels = row * col

for idx in tqdm_notebook(range(image_count)):
    label_tmp = struct.unpack('B', label_f.read(1))
    label = label_tmp[0]
    binaryData = image_f.read(pixels)
    strData = list(map(lambda x: str(x), binaryData))

    # csv에 한 줄의 데이터로 기록 -> 1 + 784개의 컬럼으로 기록 -> 1개의 데이터 표현 
    # 구분자 ','
    csv_f.write(str(label) + ',')
    csv_f.write(','.join(strData) + '\n')
    
    # pgm 파일로 dump 처리해서 확인(데이터의 유효성 확인)
    if idx == 0:    # 한 번만 수행된다.
        with open('./data/mnist/%s.pgm' % label, 'w', encoding='utf-8') as f:
            f.write('P2 28 28 255\n' + ' '.join(strData))

    # 이미지 데이터의 벡터화
    
    break
    pass

label_f.close()
image_f.close()
csv_f.close()

In [12]:
def decoding_mnist_rawData(dataStyle='train', maxCount=0):
    label_f = open('./data/mnist/{}-labels-idx1-ubyte'.format(dataStyle), 'rb')
    image_f = open('./data/mnist/{}-images-idx3-ubyte'.format(dataStyle), 'rb')
    csv_f = open('./data/mnist/{}.csv'.format(dataStyle), 'w', encoding='utf-8', newline='')

    magic_number, label_count = struct.unpack('>II', label_f.read(4 + 4))
    magic_number2, image_count, row, col = struct.unpack('>IIII', image_f.read(4 + 4 + 4 + 4))

    if maxCount > image_count:
        print('개수의 범위를 넘었습니다. 최소 %s개 이내' % image_count) 
        return
    
    elif maxCount == -1:
        maxCount = image_count
    
    elif maxCount < -1:
        print('개수의 범위를 넘었습니다. 최소 %s개 이내' % image_count)
        return
    
    pixels = row * col
    
    for idx in tqdm_notebook(range(maxCount)):
        if idx >= maxCount: break
        label_tmp = struct.unpack('B', label_f.read(1))
        label = label_tmp[0]
        binaryData = image_f.read(pixels)
        strData = list(map(lambda x: str(x), binaryData))
        csv_f.write(str(label) + ',')
        csv_f.write(','.join(strData) + '\n')

    label_f.close()
    image_f.close()
    csv_f.close()

In [13]:
# 훈련용은 750개만 준비, 테스트용은 250개만 준비한다.

# 훈련용
decoding_mnist_rawData(maxCount=30000)

# 테스트용
decoding_mnist_rawData(dataStyle='t10k', maxCount=10000)

#### [M1] 데이터 품질 향상

- 제출 : 구글드라이브 폴더 : 머신러닝_딥러닝_시험결과제출(예: 96%_이름.html)
- 점수 : 92% - 60점대, 93% - 70점대, 94% - 80점대, 95% - 85점대, 96% - 90점, 그 이상은 랭킹으로 100점까지 배치
- 정확도 96%를 목표로 머신러닝 모델을 개선한다.<br>
    [사전조치]
    - 머신러닝 모델을 이용하여 예측 시 정확도가 떨어지면 데이터의 품질, 양을 검토한다.
    - 양을 점차적으로 늘린다.
        - 데이터의 개수를 늘린다.
    - 품질을 향상시킨다.
        - 정규화
        - 차후에 적용 가능한 내용 : PCA같은 비지도 학습의 자원 축소(feature의 수를 줄인다.)
    - 비율을 조정(훈련:테스트 = 75:25)<br>
    [모델개선조치]
    - 알고리즘 교체
    - 하이퍼파라미터를 튜닝
    - 파이프라인을 이용한 전처리기를 활용(품질향상)하여 향상
    - 이런 교차 검증법을 활용하여 성능 향상을 도모한다.
    - 이런 것들의 검증은 ROC 곡선, AUC 값 등으로 확인할 수도 있고, 교차 검증법의 결과로 확인 가능

#### 데이터탐색/통찰/시각화분석

- skip

In [14]:
# 다음 함수를 만든다. load_csv(dataType='train')
# 현재 csv 파일 : t10k.csv, train.csv
# 입력 데이터 : csv 파일명
# 출력 데이터 : {'labels': [], 'images': [[]]}
def load_csv(dataType='train'):
    labels = list()
    images = list()
    
    # 1. 파일명 생성
    csv_path = './data/mnist/{}.csv'.format(dataType)

    # 2. csv 파일 오픈
    with open(csv_path, 'r') as f:
        
        # 3. 한 줄씩 읽겠다. -> 데이터의 한 세트
        for line in tqdm_notebook(f):
            tmp = line.split(',')
            labels.append(int(tmp[0]))
            images.append(list(map(lambda x: int(x), tmp[1:])))

    return {'labels': labels, 'images': images}

In [15]:
train = load_csv()
test = load_csv('t10k')

In [16]:
print(min(train['images'][0]), max(train['images'][0]))

# 값의 거리가 너무 멀다!! -> 학습 효과가 떨어진다. -> 0 ~ 1 사이로 값을 재배치 하는 것이 합리적이다.
# 정규화 처리
# 0:0, 255:1 이다. <- min max 스케일러
# 값/256 <- 값의 총 개수 = 256 <- 범주형(분류형) 값을 본다.
# 픽셀 데이터가 앞 뒤 영향을 받는 연속적 성향을 가졌는가? 독립적인 성향을 가졌는가?

0 255


In [ ]:
def load_csv_ex(dataType='train'):
    labels = list()
    images = list()
    csv_path = './data/mnist/{}.csv'.format(dataType)

    with open(csv_path, 'r') as f:
        for line in tqdm_notebook(f):
            tmp = line.split(',')
            labels.append(int(tmp[0]))
            images.append(list(map(lambda x: int(x) / 256, tmp[1:])))

    return {'labels': labels, 'images': images}

train = load_csv_ex()
test = load_csv_ex('t10k')

#### 데이터모델링(머신러닝모델링)

- 지도학습 데이터이므로 정확도를 통해서 평가를 1차로 수행

In [115]:
# 1. 모듈 가져오기
from sklearn import svm, model_selection, metrics

In [116]:
# 2. 알고리즘 생성
SEED = 2020    # 난수 고정을 통해서 SVC가 내부적으로 사용하는 난수 값의 패턴을 고정

clf = svm.SVC(random_state=SEED)

In [96]:
# 3. 데이터 분류(이미 위에서 완료)
len(train['images']), len(train['labels'])

In [97]:
# 4. 학습
clf.fit(train['images'], train['labels'])

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [98]:
# 5. 예측
pred = clf.predict(test['images'])

In [100]:
# 6. 평가
ac_score = metrics.accuracy_score(test['labels'], pred)
print(ac_score)

0.136

In [101]:
# 7. 오차행렬(혼돈행렬)을 이용한 평가
clf_report = metrics.classification_report(test['labels'], pred)
print(clf_report)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.00      0.00      0.00        19
           1       0.14      1.00      0.24        34
           2       0.00      0.00      0.00        24
           3       0.00      0.00      0.00        23
           4       0.00      0.00      0.00        33
           5       0.00      0.00      0.00        25
           6       0.00      0.00      0.00        22
           7       0.00      0.00      0.00        29
           8       0.00      0.00      0.00        14
           9       0.00      0.00      0.00        27

    accuracy                           0.14       250
   macro avg       0.01      0.10      0.02       250
weighted avg       0.02      0.14      0.03       250



#### 파이프라인 구축 및 하이퍼파라미터 튜닝

In [21]:
# 파이프라인 구축 및 하이퍼파라미터 튜닝
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

In [18]:
pipe = Pipeline([
    ('preprocessing', StandardScaler()),
    ('classifier', SVC())
])

In [19]:
param_grid = [
    {
        'classifier': [SVC()],
        'preprocessing' : [StandardScaler()],
        'classifier__C': [0.01, 0.1, 1, 10, 100],
        'classifier__gamma': [0.01, 0.1, 1, 10, 100],
        'classifier__random_state': [0]
    }
]

In [20]:
grid = GridSearchCV(pipe, param_grid, cv=5)

In [ ]:
tqdm(grid.fit(train['images'], train['labels']))

In [ ]:
grid.best_score_

In [ ]:
grid.best_params_

In [ ]:
grid.score(test['images'], test['labels'])

In [ ]:
# 실험 단계를 한 개의 process로 정리

res_ac_scores = list()
res_clf_reports = list()

# 75:25 비율을 유지한다.
for cnt in tqdm_notebook(range(1, 5), desc='count'):
    # 1. csv 저장
    decoding_mnist_rawData(maxCount=7500 * cnt)
    decoding_mnist_rawData(dataStyle='t10k', maxCount=2500 * cnt)

    # 2. csv 로드
    train = load_csv_ex()
    test = load_csv_ex('t10k')

    # 3. 모델 생성 및 학습 예측 평가
    SEED = 2020
    clf = svm.SVC(random_state=SEED)
    
    clf.fit(train['images'], train['labels'])
    
    pred = clf.predict(test['images'])
    
    ac_score = metrics.accuracy_score(test['labels'], pred)
    res_ac_scores.append(ac_score)
    
    clf_report = metrics.classification_report(test['labels'], pred)
    res_clf_reports.append(clf_report)
    
print('res_ac_scores :', res_ac_scores)
print('res_clf_reports :', res_clf_reports)